# This is a notebook for inference using WebUI + ControlNet API. 

In [ ]:
import requests
import cv2
from PIL import Image
from base64 import b64encode
import os
import base64
from tqdm import tqdm
import matplotlib.pyplot as plt
import io
import numpy as np
import torchvision.transforms as transforms
import random
import glob
import shutil


In [ ]:
url = "http://127.0.0.1:7860"

output_dir = '/path/to/output'
input_dir = '/path/to/input'
outpaint = True 

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)
random.seed(420420)
source_imgs = random.sample(os.listdir(input_dir),4613)

print('N IMAGES TO GENERATE: ', len(source_imgs))


In [ ]:
t = transforms.CenterCrop((int(1024/1024*512), 1024/1024*512))

for source_img in tqdm(source_imgs):

    tex = cv2.imread(input_dir + '/' +source_img) 

    if outpaint:
        height_margin = (max(tex.shape[:2]) - tex.shape[0])/2
        width_margin = (max(tex.shape[:2]) - tex.shape[1])/2
        add_height, add_width = int(max(0, height_margin)), int(max(0, width_margin))
        tex = cv2.copyMakeBorder(tex, add_height, add_height, add_width, add_width, cv2.BORDER_CONSTANT, value = (0,0,0))
        tex = cv2.resize(tex, (512, 512), cv2.INTER_NEAREST )
   
    retval, tex= cv2.imencode('.jpg', tex)
    tex = base64.b64encode(tex).decode('utf-8')
    lora_name_1="CholectL45"
    lora_name_2 = "CholectG45"
    
    payload = {
    "sd_model_checkpoint": "v1-5-pruned.safetensors [1a189f0be6]",
    "init_images": [tex],
    "denoising_strength": 0.65,
    "sampler_name": "DPM++ 2M Karras",
    "width": 512,
    "height": 512,
    "steps": 20,
    "cfg_scale": 7.0,
    "seed" : 420420,
    "refiner_checkpoint" : "v1-5-pruned.safetensors [1a189f0be6]",
    "refiner_switch_at" : 0.8,
    "prompt" :f"<lora:{lora_name_1}:0.45>cholect45 style,<lora:{lora_name_2}:0.45>cholect45 style",
    "negative_prompt": "",
    "include_init_images": True,
    "batch_size":1,
    "alwayson_scripts": {
      "controlnet": {
        "args": [
          {
            "model":"controlnet++_hed_softedge_sd15_fp16 [0d417fb5]",
            "enabled":True,
	          "input_image":tex,
            "module": "softedge_hedsafe",
            "weight": 0.45,
            "pixel_perfect" : True,
            "processor_res":512,
            "guidance_end": 1,
            "control_mode":0,
          },
          {
            "model":"None",
            "enabled":True,
	          "input_image":tex,
            "module": "reference_only",
            "weight": 0.65,
            "pixel_perfect" : True,
            "processor_res":512,
            "guidance_end": 1,
            "control_mode":2,
          },
          {
            "model": "controlnet++_depth_sd15_fp16 [765189fe]",
            "enabled":True,
	          "input_image":tex,
            "module": "depth_midas",
            "weight":0.65,
            "pixel_perfect" : True,
            "processor_res": 512,
            "threshold_a": 1.25,
            "control_mode":0,
          },
        ]
    }
    }
    }

    response = requests.post(url=f'{url}/sdapi/v1/img2img', json=payload)
    r = response.json()
    output = Image.open(io.BytesIO(base64.b64decode(r['images'][0].split(",",1)[0])))
    if outpaint:
      output = t(output)
    output.save(output_dir+'/'+source_img)